<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0


    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:


      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-23 10:23:01
-------------------
qualified stocks: 95
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  1.95 L
Current:  1.28 C
-------------------
Today PnL: 23.41 K (0.18%)
Current PnL: -17.01 L (-12.06%)
CY Booked + Current PnL: -13.15 L (-9.32%)
-------------------
Total profit:  3.88 L
Total loss:  -20.90 L
-------------------
Total Booked + Current PnL: 13.83 L (12.13%)
Total Booked PnL: 30.84 L (27.04%)
Curr Year Booked PnL: 3.86 L (3.03%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.08 C
Est FTT PnL: 80.25 L (62.94%)
-------------------
Est LTT:  2.69 C
Est LTT PnL: 1.41 C (110.89%)
Deployed:  1.14 C
Current:  1.28 C
CAGR/XIRR %: 7.94%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RELIANCE,1291.83,1526.00,4.23,X-LC,83.16,154758.0,13949.0,11576.0,0.74,9.91,7.48,18.13,161.0,1.20,1.23,22.20,XY25,NTT,REFINERIES
69,SBILIFE,1496.49,1924.99,0.75,H-LC,85.26,149764.0,24059.0,11936.0,1.34,19.14,7.97,28.63,159.0,2.02,1.19,33.42,AR,ATH,INSURANCE
4,ADANIPORTS,1103.69,1583.00,-2.22,M-LC,90.53,98236.0,19874.0,14156.0,0.89,25.36,14.41,43.43,48.0,1.40,0.78,31.39,XY24,NTT,MISC
56,PIDILITIND,2504.06,3576.00,-18.67,X-LC,26.32,90105.0,14983.0,17174.0,0.30,19.95,19.06,42.81,121.0,0.87,0.72,13.00,X40,BTT,CHEMICALS
70,SBIN,760.30,863.00,-14.39,M-LC,31.05,208634.0,7154.0,20071.0,0.26,3.55,9.62,13.51,60.0,0.36,1.66,16.60,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VBL,500.38,672.28,-14.71,H-LC,1.05,146736.0,-5880.0,58313.0,2.46,-3.85,39.74,34.35,23.0,-0.10,1.17,10.54,AR,ATH,FMCG
41,INFY,1432.81,2275.00,-11.70,X-LC,70.53,219954.0,20793.0,96274.0,2.26,10.44,43.77,58.78,6.0,0.22,1.75,14.84,X40,BTT,IT
37,ICICIPRULI,600.83,791.05,-11.82,H-MC,56.84,142550.0,6762.0,36222.0,1.78,4.98,25.41,31.66,175.0,0.19,1.14,17.55,X40,ATH,INSURANCE
50,LTIM,5664.39,7197.33,-5.40,X-LC,93.68,151938.0,-17994.0,63981.0,1.74,-10.59,42.11,27.06,32.0,-0.28,1.21,25.96,X200,ATH,IT
15,BSOFT,628.85,844.81,-20.22,M-SC,69.47,74935.0,-35114.0,72904.0,1.61,-31.91,97.29,34.34,19.0,-0.48,0.60,23.37,XR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.0,-55.73,L-SC,95.79,64339.0,-49210.0,89257.0,-3.73,-43.34,138.73,35.27,181.0,-0.55,0.51,57.47,XR,NTT,HOTELS
74,SIS,477.00,528.0,2074.05,M-SC,51.58,58854.0,-22236.0,30904.0,-2.60,-27.42,52.51,10.69,164.0,-0.72,0.47,19.28,X40N,NTT,MISC
88,VAIBHAVGBL,350.21,521.0,57.58,X-SC,63.16,105080.0,-46911.0,121031.0,-2.60,-30.86,115.18,48.77,57.0,-0.39,0.84,20.70,XR,NTT,JEWELLERY
84,TRIDENT,37.35,48.0,65.23,M-SC,81.05,79410.0,-12844.0,39149.0,-2.19,-13.92,49.30,28.51,207.0,-0.33,0.63,33.85,XR,NTT,TEXTILES
89,VALIANTORG,512.64,838.0,-330.82,H-SC,68.42,98228.0,-35058.0,119652.0,-1.68,-26.30,121.81,63.47,145.0,-0.29,0.78,64.65,XR,NTT,CHEMICALS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,2327.09,4911.36,-51.47,H-SC,58.95,126204.0,2868.0,134104.0,-0.48,2.33,106.26,111.05,1.0,0.02,1.01,23.66,AR,ATH,MISC
82,TCS,3888.13,4998.00,-15.52,X-LC,54.74,211626.0,-21662.0,88248.0,1.44,-9.29,41.70,28.55,3.0,-0.25,1.69,9.15,X40,BTT,IT
81,TATAMOTORS,771.32,1065.00,-52.64,M-LC,75.79,209788.0,-16209.0,102251.0,-0.20,-7.17,48.74,38.07,4.0,-0.16,1.67,23.50,XY24,NTT,AUTO
9,AWL,342.88,485.00,-69.72,H-MC,4.21,163957.0,-53772.0,144020.0,0.27,-24.70,87.84,41.45,4.0,-0.37,1.31,9.52,XY24,NTT,FMCG
41,INFY,1432.81,2275.00,-11.70,X-LC,70.53,219954.0,20793.0,96274.0,2.26,10.44,43.77,58.78,6.0,0.22,1.75,14.84,X40,BTT,IT


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4236.33,6318.3,1.50,H-SC,98.95,158100.0,26774.0,37770.0,-0.77,20.39,23.89,49.15,94.0,0.71,1.26,30.31,X40,ATH,PHARMA
4,ADANIPORTS,1103.69,1583.0,-2.22,M-LC,90.53,98236.0,19874.0,14156.0,0.89,25.36,14.41,43.43,48.0,1.40,0.78,31.39,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,44.74,M-SC,84.21,150052.0,12474.0,81943.0,0.77,9.07,54.61,68.62,66.0,0.15,1.20,30.09,X40N,NTT,TEXTILES
57,POLYCAB,5005.95,7541.35,6.25,H-MC,66.32,100836.0,15735.0,27367.0,-0.28,18.49,27.14,50.65,56.0,0.57,0.80,26.75,X40N,ATH,CABLES
55,PFIZER,4236.33,6318.30,1.50,H-SC,98.95,158100.0,26774.0,37770.0,-0.77,20.39,23.89,49.15,94.0,0.71,1.26,30.31,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,0.94,H-SC,64.21,148937.0,11077.0,26690.0,0.41,8.03,17.92,27.39,64.0,0.42,1.19,29.09,X40,ATH,FMCG
4,ADANIPORTS,1103.69,1583.00,-2.22,M-LC,90.53,98236.0,19874.0,14156.0,0.89,25.36,14.41,43.43,48.0,1.40,0.78,31.39,XY24,NTT,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,-3.78,H-SC,9.47,108077.0,6073.0,38951.0,-0.58,5.95,36.04,44.14,31.0,0.16,0.86,23.16,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-33.09,M-MC,31.05,132406.0,3254.0,83813.0,-0.09,2.52,63.30,67.42,57.0,0.04,1.05,28.78,XR,NTT,BANKS
94,WIPRO,243.46,420.00,-9.71,M-LC,48.42,154566.0,3621.0,105831.0,1.37,2.40,68.47,72.51,49.0,0.03,1.23,16.38,XR,NTT,IT
38,INDIAMART,2327.09,4911.36,-51.47,H-SC,58.95,126204.0,2868.0,134104.0,-0.48,2.33,106.26,111.05,1.0,0.02,1.01,23.66,AR,ATH,MISC
33,HEROMOTOCO,4311.81,6039.03,-1.43,H-MC,82.11,150728.0,-185.0,60638.0,0.71,-0.12,40.23,40.06,27.0,-0.00,1.20,23.29,AR,ATH,AUTO


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.0,-55.73,L-SC,95.79,64339.0,-49210.0,89257.0,-3.73,-43.34,138.73,35.27,181.0,-0.55,0.51,57.47,XR,NTT,HOTELS
52,MASFIN,326.60,399.5,-24.12,H-SC,61.05,86370.0,-11610.0,33477.0,-0.29,-11.85,38.76,22.32,102.0,-0.35,0.69,25.27,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,-2.22,M-LC,90.53,98236.0,19874.0,14156.0,0.89,25.36,14.41,43.43,48.0,1.40,0.78,31.39,XY24,NTT,MISC
89,VALIANTORG,512.64,838.0,-330.82,H-SC,68.42,98228.0,-35058.0,119652.0,-1.68,-26.30,121.81,63.47,145.0,-0.29,0.78,64.65,XR,NTT,CHEMICALS
86,UJJIVANSFB,52.77,60.0,37.69,M-SC,27.37,119205.0,-23274.0,42795.0,-0.70,-16.34,35.90,13.70,213.0,-0.54,0.95,40.69,X40N,NTT,BANKS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,VIPIND,488.80,718.00,-845.76,H-SC,100.00,71000.0,-24316.0,69012.0,-0.45,-25.51,97.20,46.89,173.0,-0.35,0.57,40.04,X40N,NTT,MISC
55,PFIZER,4236.33,6318.30,1.50,H-SC,98.95,158100.0,26774.0,37770.0,-0.77,20.39,23.89,49.15,94.0,0.71,1.26,30.31,X40,ATH,PHARMA
79,TANLA,917.30,1963.11,-40.21,H-SC,97.89,146225.0,-93190.0,366147.0,-0.34,-38.92,250.40,114.01,20.0,-0.25,1.16,34.11,AR,ATH,IT
93,WHIRLPOOL,1167.49,2270.00,-35.67,M-SC,96.84,187543.0,15922.0,146152.0,0.69,9.28,77.93,94.43,28.0,0.11,1.49,38.42,X40,NTT,DURABLES
20,COFFEEDAY,59.14,80.00,-55.73,L-SC,95.79,64339.0,-49210.0,89257.0,-3.73,-43.34,138.73,35.27,181.0,-0.55,0.51,57.47,XR,NTT,HOTELS


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.24
1,25,38.71
2,50,68.42


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.06
X40     13.88
X40N    13.82
XY25    13.68
AR      12.31
XR       9.92
X200     1.21
SR       1.13
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.72
M-SC    18.23
X-LC    15.79
H-MC     8.44
M-LC     7.79
H-LC     6.96
X-MC     5.67
M-MC     5.61
X-SC     2.83
L-SC     2.31
L-LC     1.18
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,12.17,-1.91,34.77
BANKS,10.96,-9.53,63.25
IT,9.36,-19.56,85.84
MISC,6.57,-31.06,89.03
INSURANCE,6.42,1.69,27.57
FINANCE,6.07,-25.49,85.08
PAINTS,5.54,-18.27,53.28
AUTO,4.52,-10.14,58.25
ELECTRICAL,4.35,-6.14,60.14


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2999329.0,27
AR,1225798.0,13
X40N,1215074.0,17
XR,1104526.0,14
X40,736663.0,11
XY25,476403.0,10
SR,203340.0,2
X200,63981.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2341174.0,21
M-SC,2029530.0,21
X-LC,711388.0,12
M-MC,562978.0,5
H-MC,466429.0,8
X-SC,354303.0,4
M-LC,326882.0,6
X-MC,323057.0,5
L-SC,306859.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      922875.0      6
M-SC       XY24      759233.0      6
H-SC       AR        560375.0      3
           X40N      509255.0      6
M-SC       X40N      462108.0      6
M-MC       XY24      395712.0      3
X-LC       X40       392373.0      5
M-SC       XR        296975.0      4
H-SC       XR        284209.0      4
H-LC       AR        223042.0      4
M-SC       X40       215128.0      2
M-LC       XY24      200980.0      4
H-MC       XY24      183006.0      2
L-SC       XR        154855.0      2
X-MC       XY24      144207.0      2
X-LC       AR        140760.0      2
X-SC       XR        121031.0      1
           SR        119158.0      1
M-SC       XY25      118983.0      1
X-LC       XY25      114274.0      4
X-SC       X40N      114114.0      2
M-LC       XR        105831.0      1
X-MC       X40N      102230.0      2
M-SC       AR         92921.0      1
L-SC       XY24       87395.0      1
M-SC       SR         84182.0      1
M-MC       XR         83813.0      1
           AR         83453.0      1
X-MC       XY25       76620.0      1
H-MC       XY25       72904.0      1
           X40        64702.0      2
L-SC       AR         64609.0      1
H-SC       X40        64460.0      2
X-LC       X200       63981.0      1
H-MC       AR         60638.0      1
           XR         57812.0      1
L-LC       XY25       41221.0      1
H-LC       XY25       32330.0      1
H-MC       X40N       27367.0      1
M-LC       XY25       20071.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 43.0 seconds
